<a href="https://colab.research.google.com/github/aneena-manoj/Youtube-Summarizer/blob/main/youtube_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
import re
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [10]:
link = "https://www.youtube.com/watch?v=Y8Tko2YC5hA"
unique_id = link.split("=")[-1]
sub = YouTubeTranscriptApi.get_transcript(unique_id)
subtitle = " ".join([x['text'] for x in sub])

In [12]:
import nltk

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
subtitle = subtitle.replace("n","")
sentences = sent_tokenize(subtitle)

In [15]:
print(subtitle)

I this video, I'm goig to aswer the top 3 questios my studets ask me about Pytho. What is Pytho? What  ca you do with it? Ad why is it so popular? I other words, what does it do that other programmig laguages do't? Pytho is the  world's fastest growig ad most popular programmig laguage, ot just  amogst software egieers, but also amogst mathematicias,  data aalysts, scietists, accoutats, etworkig egieers, ad eve kids! Because it's a very begier friedly programmig  laguage. So people from differet disciplies use Pytho for a variety of differet tasks, such as data aalysis ad visualizatio,  artificial itelligece ad machie learig, automatio  i fact this is oe of the big uses of Pytho amogst people who are ot software developers. If you costatly have to do borig, repetitive  tasks, such as copyig files ad folders aroud, reamig them,  uploadig them to a server, you ca easily write a Pytho script to automate all that ad save your time. Ad that's just oe example, if you cotiuously have to work 

In [16]:
print(sentences)

["I this video, I'm goig to aswer the top 3 questios my studets ask me about Pytho.", 'What is Pytho?', 'What  ca you do with it?', 'Ad why is it so popular?', "I other words, what does it do that other programmig laguages do't?", "Pytho is the  world's fastest growig ad most popular programmig laguage, ot just  amogst software egieers, but also amogst mathematicias,  data aalysts, scietists, accoutats, etworkig egieers, ad eve kids!", "Because it's a very begier friedly programmig  laguage.", 'So people from differet disciplies use Pytho for a variety of differet tasks, such as data aalysis ad visualizatio,  artificial itelligece ad machie learig, automatio  i fact this is oe of the big uses of Pytho amogst people who are ot software developers.', 'If you costatly have to do borig, repetitive  tasks, such as copyig files ad folders aroud, reamig them,  uploadig them to a server, you ca easily write a Pytho script to automate all that ad save your time.', "Ad that's just oe example, if

In [17]:
organized_sent = {k:v for v,k in enumerate(sentences)}

In [18]:
tf_idf = TfidfVectorizer(min_df=2,
                                    strip_accents='unicode',
                                    max_features=None,
                                    lowercase = True,
                                    token_pattern=r'w{1,}',
                                    ngram_range=(1, 3),
                                    use_idf=1,
                                    smooth_idf=1,
                                    sublinear_tf=1,
                                    stop_words = 'english')

In [20]:
import numpy as np

In [21]:
sentence_vectors = tf_idf.fit_transform(sentences)
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

In [22]:
N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]

In [23]:
# mapping the scored sentences with their indexes as in the subtitle
mapped_sentences = [(sentence,organized_sent[sentence]) for sentence in top_n_sentences]
# Ordering the top-n sentences in their original order
mapped_sentences = sorted(mapped_sentences, key = lambda x: x[1])
ordered_sentences = [element[0] for element in mapped_sentences]
# joining the ordered sentence
summary = " ".join(ordered_sentences)

In [29]:
print(summary)

You ca also use Pytho to build  web, mobile ad desktop applicatios as well as software  testig or eve hackig. Let's say we wat to extract the first three  letters of the text Hello World. It's cross platform which meas  you ca build ad ru Pytho applicatios o Widows, Mac,  ad Liux.


In [24]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [25]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [26]:
input_tensor = tokenizer.encode( subtitle, return_tensors="pt", max_length=512)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [27]:
outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
outputs_tensor

tensor([[    2,     0,   510, 20436,   139,    16,     5,  1437,   232,    18,
          6273,  1733,  1023,  2329,   144,  1406,   586,   119,  1023,   784,
         11993,  1580,     4,    85,    16,   341,    30,  2257,  5485,     6,
         45324,  5003,     6,   414,    10,  7776,  5019,     6,  2850,  5810,
          1952,     6,  7678,   995,  2923,     6,  4400,  6014,  1023,   364,
         25004,   268,     6,  2329, 15330,  1159,     4,   590, 46439,   139,
            47,    64,  6136,  2632,  1272,   939,   540,    86,    19,  4163,
          5738,     9,  3260,     4,   370,    64,    67,   304, 46439,   139,
             7,  1119,  1437,  3748,     6,  1830,  2329, 14050, 40967,   415,
          4544,    25,   157,    25,  2257,  1437,  1296,  1023,    50, 15330,
         14157,  1023,     4,    85,    18,    10,   182, 21422,   906, 16708,
           352, 28644,   119,  1023,  1437,   784, 11993,  3443,     4,   407,
            82,    31, 47850,    90, 41352,   918,  

In [28]:
print(tokenizer.decode(outputs_tensor[0]))

</s><s>Pytho is the  world's fastest growig ad most popular programmig laguage. It is used by software developers, mathematicias, data aalysts, scietists, accoutats, etworkig egieers, ad eve kids. With Pytho you can solve complex problems i less time with fewer lies of code. You can also use Pytho to build  web, mobile ad desktop applicatios as well as software  testig or eve hackig. It's a very begier friedlyprogrammig  laguages. So people from differet disciplies use it for a variety of tasks.</s>
